# Simulate Data


Given a field name, and first 2 dimensional numbes, simulate tensor.

In [1]:
# from IPython.core.alias import Instance
import numpy as np

def traverse(o, tree_types=(list, tuple, np.ndarray), index = None, nest_layer = 100):
    if isinstance(o, tree_types) and nest_layer > 0:
        for idx, value in enumerate(o):
            new_index = index + [idx] if type(index) == list else [idx]
            for subvalue in traverse(value, tree_types, new_index, nest_layer - 1):
                yield subvalue
    else:
        if not isinstance(o, tree_types): 
            length = None
        else:
            length = len(o)
        yield index, length, o

In [4]:
# TODO: based on fld_name, get input_tensor
import numpy as np
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 100

###############
# 'B-P-EC:Diag-Rec:V-VdftGrn'
# B: batch
# P: patient
# EC:Diag: Encounter's DiagInfo
# Rec:V: a Rec's Value: E11
# VdftGrn: Value to a list of Grn. ICD-10 [E, 1, 1]
fld_name = 'B-P-EC:Diag-Rec:V-VdftGrn'
###############


# fld_name = 'B2St2TkGrn'
layers_num = len(fld_name.split('-'))
print(layers_num)

# max_list = df['max'].astype(int).to_list()
prefix = [np.array(B_lenP).max(), max(B2P_lnEC)] 
max_list = [np.array(B_lenP).max(), max(B2P_lnEC)] + list(np.random.randint(1, 10, layers_num - len(prefix) - 1)) + [vocab_size]
print(max_list)

5
[3, 6, 8, 4, 100]


In [5]:
def get_next_info(layer, current_info, max_list):
    current_max = max_list[layer]
    next_max = max_list[layer + 1]
    # to do: get next_info
    next_info = np.zeros(list(np.array(current_info).shape) + [current_max]).astype(int)
    # print(next_info.shape)
    for element in list(traverse(current_info, nest_layer = layer)):
        idx, leng, value = element
        # print(next_info[tuple(idx)])
        next_info[tuple(idx)][:value] = np.random.randint(1, next_max + 1, value)

    # print(next_info)
    return next_info

In [6]:
def get_simulated_tensor_from_fldname(fld_name, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size):
    layers_num = len(fld_name.split('-'))
    # print(layers_num)

    # max_list = df['max'].astype(int).to_list()
    prefix = [np.array(B_lenP).max(), max(B2P_lnEC)] 
    max_list = [np.array(B_lenP).max(), max(B2P_lnEC)] + list(np.random.randint(1, 10, layers_num - len(prefix) - 1)) + [vocab_size]
    # print(max_list)

    init_info = np.array(B2P_lnEC)
    
    for layer_idx in range(prefix_layers_num - 1, layers_num - 1):
        print(layer_idx)
        current_info = init_info
        next_info = get_next_info(layer_idx, current_info, max_list)
        # print(next_info)
        init_info = next_info

        print(layer_idx, '-->', current_info.shape)
        print(layer_idx + 1, '-->', next_info.shape)


    fld_tensor_idx = next_info
    # print(fld_tensor.shape)
    return fld_tensor_idx


In [15]:
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 100


fld_name = 'B-P-EC:Diag-DiagRec:V-VdftGrn'
fld_tensor_idx = get_simulated_tensor_from_fldname(fld_name, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(fld_tensor_idx.shape)
# fld_tensor_idx

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 4)
3
3 --> (3, 6, 4)
4 --> (3, 6, 4, 3)
(3, 6, 4, 3)


In [23]:
fld_tensor_idx.shape

# ( 3,6,      4,    3)
# 'B-P-EC:Diag-DiagRec:V-VdftGrn' [E, 1, 1]
# fld_tensor_idx

(3, 6, 4, 3)

# info_idx, info, leng_mask, leng

1. Current Layer:
```python
leng_mask = info_idx == 0 # or info_idx != 0
leng = leng_mask.sum(-1)
```

2. Transfer

```python
old_leng = leng
```


3. Next Layer
```python
leng_mask = old_leng != 0
leng = (leng_mask == 0).sum(-1)
```

In [55]:
########################
B_lenP = 3
B2P_lnEC = [6, 4, 3] # 
prefix_layers_num = 2
vocab_size = 100
########################

fullname = 'B-P-EC:Diag-DiagRec:V-VdftGrn'
# fld_name = 'B2St2TkGrn'
data = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(data.shape)
# fld_tensor_idx

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 7)
3
3 --> (3, 6, 7)
4 --> (3, 6, 7, 6)
(3, 6, 7, 6)


## 'B-P-EC:Diag-DiagRec:V-VdftGrn' 

In [56]:
data

array([[[[ 50,  23,  55,   0,   0,   0],
         [ 60,  76,  45,  54,   0,   0],
         [ 84,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0]],

        [[ 17,   7,  69,  97,  92,   0],
         [  0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0]],

        [[ 25,  71,  38,   0,   0,   0],
         [ 12,  99,  82,  72,   0,   0],
         [ 87,  36,  98,   0,   0,   0],
         [ 17,  81,  33,   0,   0,   0],
         [ 34,  44,   0,   0,   0,   0],
         [ 16,  81,  17,  85,   0,   0],
         [  0,   0,   0,   0,   0,   0]],

        [[ 25,  71,  13,  84,  56,  15],
         [ 13,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0],
         [

In [57]:
import torch
info_idx = torch.LongTensor(data)
print(info_idx.shape)

holder = info_idx

leng_mask = holder == 0
print(leng_mask.shape)
leng = (leng_mask == 0).sum(-1)
pos_idx = (leng_mask == False).cumsum(-1).masked_fill(leng_mask, 0)
print(pos_idx.shape)

leng_mask


torch.Size([3, 6, 7, 6])
torch.Size([3, 6, 7, 6])
torch.Size([3, 6, 7, 6])


tensor([[[[False, False, False,  True,  True,  True],
          [False, False, False, False,  True,  True],
          [False,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True]],

         [[False, False, False, False, False,  True],
          [ True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True]],

         [[False, False, False,  True,  True,  True],
          [False, False, False, False,  True,  True],
          [False, False, False,  True,  True,  True],
          [False, False, False,  True,  True,  True],
          [False, False,

## 'B-P-EC:Diag-DiagRec:V' 

In [58]:
# next layer
holder = leng

leng_mask = holder == 0
print(leng_mask.shape)
leng = (leng_mask == 0).sum(-1)

leng_mask

torch.Size([3, 6, 7])


tensor([[[False, False, False,  True,  True,  True,  True],
         [False,  True,  True,  True,  True,  True,  True],
         [False, False, False, False, False, False,  True],
         [False, False,  True,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True],
         [False, False, False, False, False, False,  True]],

        [[False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True],
         [False, False, False, False, False, False,  True],
         [False, False, False, False, False,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True]],

        [[False, False, False, False,  True,  True,  True],
         [False, False, False, False, False, False,  True],
         [False, False, False,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True

## 'B-P-EC:Diag' 

In [59]:
holder = leng
leng_mask = holder == 0
print(leng_mask.shape)
leng = (leng_mask == 0).sum(-1)

leng_mask

torch.Size([3, 6])


tensor([[False, False, False, False, False, False],
        [False, False, False, False,  True,  True],
        [False, False, False,  True,  True,  True]])

## 'B-P' 

In [61]:
holder = leng
leng_mask = holder == 0

print(leng_mask.shape)
leng_mask

torch.Size([3])


tensor([False, False, False])

In [62]:
leng = (leng_mask == 0).sum(-1)
leng

tensor(3)

In [63]:
B2P_lnEC

[6, 4, 3]

# Holder

In [65]:
########################
B_lenP = 3
B2P_lnEC = [6, 4, 3] # 
prefix_layers_num = 2
vocab_size = 100
########################

fullname = 'B-P-EC:Diag-DiagRec:DiagV-DiagVdftGrn'
Ignore_PSN_Layers = ['B', 'P']
# fullname = 'B2St2TkGrn'
info_idx = get_simulated_tensor_from_fldname(fld_name, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 5)
3
3 --> (3, 6, 5)
4 --> (3, 6, 5, 5)


In [66]:
Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
name = fullname.split('-')[-1]
print(Layer2Idx)
print(name)

{'B': 0, 'P': 1, 'EC:Diag': 2, 'DiagRec:DiagV': 3, 'DiagVdftGrn': 4}
DiagVdftGrn


In [67]:
import torch

holder = info_idx
holder = torch.LongTensor(holder)

info = None
print(fullname)
print(name)
print(holder.shape, '<-- holder')

B-P-EC:Diag-DiagRec:DiagV-DiagVdftGrn
DiagVdftGrn
torch.Size([3, 6, 5, 5]) <-- holder


In [68]:
# leng_mask = info_idx == 0

def get_Layer2Holder(fullname, holder, Ignore_PSN_Layers = ['B', 'P']):
    # holder = holder
    d = {}
    for layername in list(reversed(fullname.split('-'))):
        if layername in Ignore_PSN_Layers: continue
        leng_mask = holder == 0
        leng = (leng_mask == 0).sum(-1)
        psn_idx = (leng_mask == False).cumsum(-1).masked_fill(leng_mask, 0)
        d[layername] = {'holder': holder, 
                        'leng_mask': leng_mask, 
                        'leng': leng, 
                        'psn_idx': psn_idx}
        # d[layername] = holder, psn_idx
        holder = leng
    Layer2Hoder = d
    return Layer2Hoder

print(holder.shape)
print(fullname)
Layer2Holder = get_Layer2Holder(fullname, holder)
for i in Layer2Holder: print(i)

torch.Size([3, 6, 5, 5])
B-P-EC:Diag-DiagRec:DiagV-DiagVdftGrn
DiagVdftGrn
DiagRec:DiagV
EC:Diag


In [69]:

def align_psn_idx(source_layer, current_layer, Layer2Idx, Layer2Holder):
    if source_layer == current_layer:
        psn_idx = Layer2Holder[current_layer]['psn_idx']
        return psn_idx
    else:
        source_psn_idx = Layer2Holder[source_layer]['psn_idx']
        current_leng_mask = Layer2Holder[current_layer]['leng_mask']
        gaps = Layer2Idx[current_layer] - Layer2Idx[source_layer]
        # print(gaps)
        # print(layername)
        # print(prev_info.shape)
        # print(leng_mask.shape)
        # print(leng.shape)
        # print(psn_idx.shape)
        shape0 = list(source_psn_idx.shape) + [1] * gaps
        shape1 = current_leng_mask.shape
        psn_idx = source_psn_idx.view(*shape0).expand(shape1).masked_fill(current_leng_mask, 0)
        # print(cpsn_idx.shape)
        return psn_idx

In [70]:
[i for i in Layer2Holder]

['DiagVdftGrn', 'DiagRec:DiagV', 'EC:Diag']

In [71]:
psn_layers = [i for i in Layer2Holder]

print(name) # 'DiagVdftGrn'
for source_layer in psn_layers:
    cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
    print(cpsn_idx.shape, source_layer)
    

DiagVdftGrn
torch.Size([3, 6, 5, 5]) DiagVdftGrn
torch.Size([3, 6, 5, 5]) DiagRec:DiagV
torch.Size([3, 6, 5, 5]) EC:Diag


In [72]:
# Layer2Holder

name = 'DiagVdftGrn'
source_layer = 'DiagRec:DiagV'
psn_idx = Layer2Holder[source_layer]['psn_idx']
cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
# cpsn_idx

# Embedding (Expander)

## Module

In [73]:
# fieldlm.nn.embedding
import os
import torch
import numpy as np

class EmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 input_size, 
                 embedding_size, 
                 init = 'init', 
                 freeze = False):
        
        super(EmbeddingLayer, self).__init__()
        
        # (+) self.embedding
        if type(init) == np.ndarray:
            # 1. from given array
            weight = torch.FloatTensor(init)
            assert weight.shape == (input_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
            
        elif os.path.isfile(init):
            weight = torch.FloatTensor(np.load(init))
            assert tuple(weight.shape) == (input_size, embedding_size)
            
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
            
        else:
            # from random initialization
            self.embedding = torch.nn.Embedding(input_size, embedding_size, padding_idx = 0)
        
    def forward(self, info):
        # tensor0 to tensor1
        info = self.embedding(info)
        return info

## Config

In [74]:
[i for i in Layer2Holder]

['DiagVdftGrn', 'DiagRec:DiagV', 'EC:Diag']

In [75]:
Layer2Holder['EC:Diag']

{'holder': tensor([[2, 2, 1, 5, 3, 2],
         [1, 4, 1, 3, 0, 0],
         [2, 1, 3, 0, 0, 0]]),
 'leng_mask': tensor([[False, False, False, False, False, False],
         [False, False, False, False,  True,  True],
         [False, False, False,  True,  True,  True]]),
 'leng': tensor([6, 4, 3]),
 'psn_idx': tensor([[1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 0, 0],
         [1, 2, 3, 0, 0, 0]])}

In [76]:
embed_size = 512
vocab_size = 5000

embed_para =  {'embedding_size': embed_size,
               'init': 'random', 
               'input_size': vocab_size + 1 } # 1:the size of special tokens
embed_para

{'embedding_size': 512, 'init': 'random', 'input_size': 5001}

In [77]:
def generate_psn_embed_para(layername, embed_size):
    
    if 'Grn' not in layername: 
        vocab_size = 100
    else:
        vocab_size = 512
        
    embed_para = {'embedding_size': embed_size,
                  'init': 'random', 
                  'input_size': vocab_size + 1 }
    return embed_para

In [78]:
d = {}
d[fullname] = embed_para



psn_layers = [i for i in Layer2Holder]

for layername in psn_layers:
    if layername == 'P': break
    embed_para = generate_psn_embed_para(layername, embed_size)
    d[f'{layername}_psn'] = generate_psn_embed_para(layername, embed_size)
    
d

{'B-P-EC:Diag-DiagRec:DiagV-DiagVdftGrn': {'embedding_size': 512,
  'init': 'random',
  'input_size': 5001},
 'DiagVdftGrn_psn': {'embedding_size': 512,
  'init': 'random',
  'input_size': 513},
 'DiagRec:DiagV_psn': {'embedding_size': 512,
  'init': 'random',
  'input_size': 101},
 'EC:Diag_psn': {'embedding_size': 512, 'init': 'random', 'input_size': 101}}

In [79]:
for nn_name, layer_para in d.items():
    print(nn_name)
    print(layer_para)

B-P-EC:Diag-DiagRec:DiagV-DiagVdftGrn
{'embedding_size': 512, 'init': 'random', 'input_size': 5001}
DiagVdftGrn_psn
{'embedding_size': 512, 'init': 'random', 'input_size': 513}
DiagRec:DiagV_psn
{'embedding_size': 512, 'init': 'random', 'input_size': 101}
EC:Diag_psn
{'embedding_size': 512, 'init': 'random', 'input_size': 101}


## Usage 

In [80]:
NN_Dict = {}

for nn_name, layer_para in d.items():
    layer = EmbeddingLayer(**layer_para)
    NN_Dict[nn_name] = layer
    
NN_Dict

{'B-P-EC:Diag-DiagRec:DiagV-DiagVdftGrn': EmbeddingLayer(
   (embedding): Embedding(5001, 512, padding_idx=0)
 ),
 'DiagVdftGrn_psn': EmbeddingLayer(
   (embedding): Embedding(513, 512, padding_idx=0)
 ),
 'DiagRec:DiagV_psn': EmbeddingLayer(
   (embedding): Embedding(101, 512, padding_idx=0)
 ),
 'EC:Diag_psn': EmbeddingLayer(
   (embedding): Embedding(101, 512, padding_idx=0)
 )}

In [81]:
import torch

########################
B_lenP = 3
B2P_lnEC = [6, 4, 3] # 
prefix_layers_num = 2
vocab_size = 100
########################

fullname = 'B-P-EC:Diag-DiagRec:DiagV-DiagVdftGrn'
# fullname = 'B2St2TkSlfGrn'
layer2layeridx = {v:idx for idx, v in enumerate(fullname.split('2'))}
name = fullname.split('-')[-1]

data = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(data.shape)
# fld_tensor_idx


info_idx = torch.LongTensor(data)
print(info_idx.shape)
holder = info_idx
# grn_leng_mask = info_idx == 0
# print(grn_leng_mask.shape)

Layer2Holder = get_Layer2Holder(fullname, holder)
for i in Layer2Holder: print(i)

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 1)
3
3 --> (3, 6, 1)
4 --> (3, 6, 1, 6)
(3, 6, 1, 6)
torch.Size([3, 6, 1, 6])
DiagVdftGrn
DiagRec:DiagV
EC:Diag


In [82]:
# embed
Embed = NN_Dict[fullname]
Embed

EmbeddingLayer(
  (embedding): Embedding(5001, 512, padding_idx=0)
)

In [83]:
info = Embed(info_idx)
# tensor_name = tensor_name.replace('_idx', '2Feat_flt')
print(info.shape)
# print(tensor_name)

torch.Size([3, 6, 1, 6, 512])


In [84]:
# tensor_name.replace('_idx', '2Feat_flt')

# Transformer (Learner)

## Module

In [85]:
import torch
import numpy as np
import torch.nn.functional as F

class TFMLayer(torch.nn.Module):
    def __init__(self, 
                 input_size = 512, 
                 output_size = 512, # d_model
                 nhead = 8,
                 num_encoder_layers = 6, # only have encoder part
                 num_decoder_layers = 0, # in default, we don't need decoder part. 
                 dim_feedforward = 2048, 
                 tfm_dropout = 0.1,
                 tfm_activation = 'relu'):
        
        '''https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/transformer.py'''

        super(TFMLayer,self).__init__()
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.input_size = input_size
        self.tfm_input_size = input_size
        self.n_directions = 1
        self.output_size = output_size
        assert output_size % self.n_directions == 0 
        self.hidden_size = int(output_size / self.n_directions)
        assert self.hidden_size == self.tfm_input_size
            
        self.transformer  = torch.nn.Transformer(d_model = self.hidden_size, 
                                                 nhead = nhead,
                                                 num_encoder_layers = self.num_encoder_layers,
                                                 num_decoder_layers = self.num_decoder_layers,
                                                 dim_feedforward = dim_feedforward, 
                                                 dropout = tfm_dropout,
                                                 activation = tfm_activation,
                                                 batch_first = True,
                                                 # src_mask_flag = False, # see all tokens in a sentence 
                                                 # # This IS THE NEW PART. NOT PyTorch.nn.
                                                 ) 
        # self.postprocess = []
        # for method, use_config in postprecess.items():
        #     use, config = use_config
        #     if use == False: continue
        #     if method == 'activator':
        #         activator = config
        #         if activator.lower() == 'relu': 
        #             self.activator = F.relu
        #         elif activator.lower() == 'tanh': 
        #             self.activator = F.tanh
        #         elif activator.lower() == 'gelu':
        #             self.activator = F.gelu
        #         else:
        #             self.activator = lambda x: x
        #         self.postprocess.append(self.activator)
            
        #     if method == 'dropout':
        #         self.drop = torch.nn.Dropout(**config)
        #         self.postprocess.append(self.drop)
                
        #     elif method == 'layernorm':
        #         self.layernorm = torch.nn.LayerNorm(self.output_size, **config)
        #         self.postprocess.append(self.layernorm)


    def forward(self, info, leng_mask):
        info = self.transformer(info, info, src_key_padding_mask = leng_mask,  tgt_key_padding_mask  = leng_mask)
        # for layer in self.postprocess:
        #     info = layer(info)
        return info

## Config

In [86]:
tfm_para =  {'input_size': 512,
             'output_size': 512,
             'nhead': 8,
             'num_encoder_layers': 6,
             'num_decoder_layers': 0,
             'dim_feedforward': 2048,
             'tfm_dropout': 0.1,
             'tfm_activation': 'relu'}
tfm_para

{'input_size': 512,
 'output_size': 512,
 'nhead': 8,
 'num_encoder_layers': 6,
 'num_decoder_layers': 0,
 'dim_feedforward': 2048,
 'tfm_dropout': 0.1,
 'tfm_activation': 'relu'}

## Usage

In [87]:
tfm_layer = TFMLayer(**tfm_para)
tfm_layer

TFMLayer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, o

In [88]:
print(fld_name)
tensor_name = fld_name + '_idx'
print(tensor_name)
print(info_idx.shape)
info = Embed(info_idx)
tensor_name = tensor_name.replace('_idx', '2Feat_flt')
print(info.shape)
print(tensor_name)

B-P-EC:Diag-DiagRec:V-VdftGrn
B-P-EC:Diag-DiagRec:V-VdftGrn_idx
torch.Size([3, 6, 1, 6])
torch.Size([3, 6, 1, 6, 512])
B-P-EC:Diag-DiagRec:V-VdftGrn2Feat_flt


In [89]:

print(info_idx.shape)
leng_mask = info_idx == 0
print(info.shape)
print(leng_mask.shape)

torch.Size([3, 6, 1, 6])
torch.Size([3, 6, 1, 6, 512])
torch.Size([3, 6, 1, 6])


### Reshape

In [90]:
nbs = np.array(info.shape[:-2]).prod()
ngrn, dim = info.shape[-2:]
print(nbs, ngrn, dim)


tmp_info = info.contiguous().view(nbs, ngrn, dim)
print(tmp_info.shape)

tmp_leng_mask = leng_mask.contiguous().view(nbs, ngrn)
print(tmp_leng_mask.shape)

tmp_leng = (tmp_leng_mask == 0).sum(-1)
print(tmp_leng.shape)

18 6 512
torch.Size([18, 6, 512])
torch.Size([18, 6])
torch.Size([18])


In [91]:
# order sequences and restore sequences according to their lenghts
# TODO: test the speed of orderSeq and restoreSeq

def orderSeq(seq_unordered, leng_unordered):
    # leng_unordered is a tensor
    # seq_unordered is a numpy
    leng_ordered, seq_index = leng_unordered.sort(descending=True) 
    _, reverse_index = seq_index.sort()
    leng_ordered = leng_ordered[leng_ordered>0]
    seq_index    = seq_index[:len(leng_ordered)]
    seq_ordered  = seq_unordered[seq_index.cpu()]
    return seq_ordered, leng_ordered, reverse_index

In [92]:
tmp_info = info.contiguous().view(nbs, ngrn, dim)
print(tmp_info.shape)

ord_info,      ord_leng, r_idx = orderSeq(tmp_info, tmp_leng)
ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
print(ord_info.shape)
print(ord_leng.shape)
print(r_idx.shape)
print(ord_leng)

torch.Size([18, 6, 512])
torch.Size([13, 6, 512])
torch.Size([13])
torch.Size([18])
tensor([6, 6, 6, 5, 4, 4, 3, 3, 3, 3, 2, 2, 1])


In [93]:
print(tmp_leng_mask.shape)
ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
print(ord_leng_mask.shape)
print(ord_leng.shape)
print(r_idx.shape)
print(ord_leng)

torch.Size([18, 6])
torch.Size([13, 6])
torch.Size([13])
torch.Size([18])
tensor([6, 6, 6, 5, 4, 4, 3, 3, 3, 3, 2, 2, 1])


### Calculate

In [94]:
ord_info_output = tfm_layer(ord_info, ord_leng_mask)
print(ord_info_output.shape)

torch.Size([13, 6, 512])


### Restore

In [95]:
def restoreSeq(seq_ordered, reverse_index):
    # shape = list(seq_ordered.shape)
    data_type = seq_ordered.type()
    shape = list(seq_ordered.shape)
    shape[0] = len(reverse_index) - shape[0]
    t = torch.cat([seq_ordered, torch.zeros(shape).type(data_type)])
    seq_restored = t[reverse_index]
    return seq_restored

In [96]:
info_new = restoreSeq(ord_info_output, r_idx)
print(info_new.shape)

torch.Size([18, 6, 512])


In [97]:
output_size = dim
info_output = info_new.view(*list(leng_mask.shape) + [output_size])
print(info_output.shape)

torch.Size([3, 6, 1, 6, 512])


In [98]:
info_output[:,:,:,:,0]

tensor([[[[ 0.4219, -0.1160, -0.5974,  0.4538, -0.4686,  1.8094]],

         [[ 0.9934,  0.3924,  0.2562,  1.4032,  0.3456,  1.4251]],

         [[ 0.3973,  0.2109, -0.8690,  0.0000,  0.0000,  0.0000]],

         [[ 0.3130, -0.0413, -0.7882,  0.0000,  0.0000,  0.0000]],

         [[-1.4392,  0.3513,  0.7602,  0.0945, -0.2210,  0.0000]],

         [[ 2.4675,  1.4251, -0.6225, -0.0158,  1.5988,  0.5250]]],


        [[[ 0.8921, -0.7882,  0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.3973, -0.0158, -0.8690,  0.1951,  0.0000,  0.0000]],

         [[-0.6225,  0.4470,  0.3456,  0.0000,  0.0000,  0.0000]],

         [[-0.3868,  0.1158,  0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]],


        [[[ 1.5988, -0.0158,  0.4735, -0.7329,  0.0000,  0.0000]],

         [[-0.6603, -0.0467, -0.7141,  0.0000,  0.0000,  0.0000]],

         [[ 1.4032,  0.0000,  0.0000,  0.000

In [99]:
leng_mask

tensor([[[[False, False, False, False, False, False]],

         [[False, False, False, False, False, False]],

         [[False, False, False,  True,  True,  True]],

         [[False, False, False,  True,  True,  True]],

         [[False, False, False, False, False,  True]],

         [[False, False, False, False, False, False]]],


        [[[False, False,  True,  True,  True,  True]],

         [[False, False, False, False,  True,  True]],

         [[False, False, False,  True,  True,  True]],

         [[False, False,  True,  True,  True,  True]],

         [[ True,  True,  True,  True,  True,  True]],

         [[ True,  True,  True,  True,  True,  True]]],


        [[[False, False, False, False,  True,  True]],

         [[False, False, False,  True,  True,  True]],

         [[False,  True,  True,  True,  True,  True]],

         [[ True,  True,  True,  True,  True,  True]],

         [[ True,  True,  True,  True,  True,  True]],

         [[ True,  True,  True,  True,  True